# Quick Start

See the API for more detailed information, examples, formulas, and references for each function.

In [2]:
import numpy as np
import xarray as xr
import xskillscore as xs

Here, we generate some sample gridded data. Our data has three time steps, and a 4x5 latitude/longitude grid. `obs` replicates some verification data and `fct` some forecast (e.g. from a statistical or dynamical model).

In [3]:
obs = xr.DataArray(
       np.random.rand(3, 4, 5),
       coords=[
           xr.cftime_range("2000-01-01", "2000-01-03", freq="D"),
           np.arange(4),
           np.arange(5),
       ],
       dims=["time", "lat", "lon"],
   )
fct = obs.copy()
fct.values = np.random.rand(3, 4, 5)

## Deterministic Metrics

`xskillscore` offers a suite of correlation-based and distance-based deterministic metrics.

### Correlation-Based 

* Pearson Correlation (`pearson_r`)
* Pearson Correlation p value (`pearson_r_p_value`)
* Pearson Correlation effective p value (`pearson_r_eff_p_value`)
* Spearman Correlation (`spearman_r`)
* Spearman Correlation p value (`spearman_r_p_value`)
* Spearman Correlation effective p value (`spearman_r_eff_p_value`)
* Effective Sample Size (`effective_sample_size`)
* Coefficient of Determination (`r2`)

### Distance-Based

* Root Mean Squared Error (`rmse`)
* Mean Squared Error (`mse`)
* Mean Absolute Error (`mae`)
* Median Absolute Error (`median_absolute_error`)
* Symmetric Mean Absolute Percentage Error (`smape`)
* Mean Absolute Percentage Error (`mape`)

Calling the functions is very straight-forward. All deterministic functions take the form `func(a, b, dim, **kwargs)`. **Notice that the original dataset is reduced by the dimension passed.** I.e., since we passed `time` as the dimension here, we are returned an object with dimensions `(lat, lon)`.

In [4]:
r = xs.pearson_r(obs, fct, "time")
print(r)

<xarray.DataArray (lat: 4, lon: 5)>
array([[-0.67103963, -0.7994918 , -0.4877047 ,  0.9009042 , -0.99576019],
       [ 0.01011846, -0.98098359,  0.99660269,  0.98889716,  0.16882124],
       [ 0.53017399, -0.27949033, -0.39054563, -0.62662994, -0.08972376],
       [-0.99835429,  0.44805023,  0.36821353,  0.99698985,  0.99991305]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


In [5]:
p = xs.pearson_r_p_value(obs, fct, "time")
print(p)

<xarray.DataArray (lat: 4, lon: 5)>
array([[0.53169606, 0.41020445, 0.67566856, 0.28580915, 0.05864376],
       [0.99355828, 0.12435121, 0.05249113, 0.09495434, 0.89200788],
       [0.6442532 , 0.81966902, 0.74457273, 0.56886755, 0.94280316],
       [0.03652847, 0.70423714, 0.75993907, 0.04940812, 0.00839532]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


You can also specify multiple axes for deterministic metrics. Here, we apply it over the latitude and longitude dimension (a pattern correlation).

In [6]:
r = xs.pearson_r(obs, fct, ["lat", "lon"])
print(r)

<xarray.DataArray (time: 3)>
array([ 0.12015305, -0.04756884, -0.10660076])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


All deterministic metrics except for `pearson_r_eff_p_value`, `spearman_r_eff_p_value`, and `effective_sample_size` can take the kwarg `weights=...`. `weights` should be a DataArray of the size of the reduced dimension (e.g., if time is being reduced it should be of length 3 in our example).

Weighting is a common practice when working with observations and model simulations of the Earth system. When working with rectilinear grids, one can weight the data by the cosine of the latitude, which is maximum at the equator and minimum at the poles (as in the below example). More complicated model grids tend to be accompanied by a cell area varaible, which could also be passed into this function.

In [7]:
obs2 = xr.DataArray(
       np.random.rand(3, 180, 360),
       coords=[
           xr.cftime_range("2000-01-01", "2000-01-03", freq="D"),
           np.linspace(-89.5, 89.5, 180),
           np.linspace(-179.5, 179.5, 360),
       ],
       dims=["time", "lat", "lon"],
    )
fct2 = obs2.copy()
fct2.values = np.random.rand(3, 180, 360)

In [8]:
# make weights as cosine of the latitude and broadcast
weights = np.cos(np.deg2rad(obs2.lat))
_, weights = xr.broadcast(obs2, weights)

# Remove the time dimension from weights
weights = weights.isel(time=0)

In [9]:
r_weighted = xs.pearson_r(obs2, fct2, ["lat", "lon"], weights=weights)
print(r_weighted)

<xarray.DataArray (time: 3)>
array([ 0.00962148,  0.00248914, -0.00356835])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


In [10]:
r_unweighted = xs.pearson_r(obs2, fct2, ["lat", "lon"], weights=None)
print(r_unweighted)

<xarray.DataArray (time: 3)>
array([ 0.01125428, -0.0005663 , -0.00429196])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


You can also pass the optional boolean kwarg `skipna=...`. If `True`, ignore any NaNs (pairwise) in `a` and `b` when computing the result. If `False`, return NaNs anywhere there are pairwise NaNs.

In [11]:
obs_with_nans = obs.where(obs.lat > 1)
fct_with_nans = fct.where(fct.lat > 1)
print(obs_with_nans)

<xarray.DataArray (time: 3, lat: 4, lon: 5)>
array([[[       nan,        nan,        nan,        nan,        nan],
        [       nan,        nan,        nan,        nan,        nan],
        [0.94394431, 0.00241133, 0.88655773, 0.24855886, 0.12939016],
        [0.74368656, 0.43481756, 0.09270441, 0.70121719, 0.65438619]],

       [[       nan,        nan,        nan,        nan,        nan],
        [       nan,        nan,        nan,        nan,        nan],
        [0.97952998, 0.11696087, 0.02818402, 0.41481101, 0.06008297],
        [0.48527913, 0.78267699, 0.9700981 , 0.68906662, 0.42554125]],

       [[       nan,        nan,        nan,        nan,        nan],
        [       nan,        nan,        nan,        nan,        nan],
        [0.95429095, 0.77787857, 0.11121986, 0.90936295, 0.56665484],
        [0.43491673, 0.53301838, 0.14625798, 0.14301593, 0.30789034]]])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00
  * lat      (lat) int64 

In [12]:
mae_with_skipna = xs.mae(obs_with_nans, fct_with_nans, ['lat', 'lon'], skipna=True)
print(mae_with_skipna)

<xarray.DataArray (time: 3)>
array([0.3581193 , 0.31215209, 0.38782476])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


In [13]:
mae_without_skipna = xs.mae(obs_with_nans, fct_with_nans, ['lat', 'lon'], skipna=False)
print(mae_without_skipna)

<xarray.DataArray (time: 3)>
array([nan, nan, nan])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


## Probabilistic Metrics

`xskillscore` offers a suite of probabilistic metrics, mostly ported from `properscoring`.

* Brier Score (`brier_score`)
* Discrimination (`discrimination`)
* Rank Histogram (`rank_histogram`)
* Continuous Ranked Probability Score with the ensemble distribution (`crps_ensemble`)
* Continuous Ranked Probability Score with a Gaussian distribution (`crps_gaussian`)
* Continuous Ranked Probability Score with numerical integration of the normal distribution (`crps_quadrature`)
* Brier scores of an ensemble for exceeding given thresholds (`threshold_brier_score`)


We now create some data with an ensemble member dimension. In this case, we envision an ensemble forecast with multiple members to validate against our theoretical observations.

In [14]:
obs3 = xr.DataArray(
       np.random.rand(4, 5),
       coords=[np.arange(4), np.arange(5)],
       dims=["lat", "lon"]
   )
fct3 = xr.DataArray(
       np.random.rand(3, 4, 5),
       coords=[np.arange(3), np.arange(4), np.arange(5)],
       dims=["member", "lat", "lon"],
   )

Continuous Ranked Probability Score with the ensemble distribution.

In [15]:
crps_ensemble = xs.crps_ensemble(obs3, fct3)
print(crps_ensemble)

<xarray.DataArray (lat: 4, lon: 5)>
array([[0.10467369, 0.06156941, 0.1410602 , 0.10320587, 0.03301175],
       [0.28093861, 0.12891183, 0.37212301, 0.13652023, 0.21922018],
       [0.03215846, 0.32533176, 0.20797605, 0.57981331, 0.37718452],
       [0.11517648, 0.17513359, 0.03096277, 0.61624043, 0.33855379]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


The CRPS with a Gaussian distribution requires two parameters: $\mu$ and $\sigma$ from the forecast distribution. Here, we just use the ensemble mean and ensemble spread.

In [16]:
crps_gaussian = xs.crps_gaussian(obs3, fct3.mean("member"), fct3.std("member"))
print(crps_gaussian)

<xarray.DataArray (lat: 4, lon: 5)>
array([[0.07458379, 0.03396951, 0.13939311, 0.09920366, 0.02800053],
       [0.27520707, 0.13358856, 0.36454362, 0.10009129, 0.1985102 ],
       [0.02917916, 0.31957214, 0.20334355, 0.57170464, 0.37210022],
       [0.05558333, 0.18163534, 0.022923  , 0.61590686, 0.33405069]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


The CRPS quadrature metric requires a callable distribution function. Here we use `norm` from `scipy.stats`.

In [17]:
from scipy.stats import norm
crps_quadrature = xs.crps_quadrature(obs3, norm)
print(crps_quadrature)

<xarray.DataArray (lat: 4, lon: 5)>
array([[0.31801604, 0.33631719, 0.27560464, 0.39399055, 0.39258011],
       [0.44293259, 0.43650668, 0.23669881, 0.36556428, 0.52749683],
       [0.2586594 , 0.2394579 , 0.28837827, 0.53477731, 0.2499387 ],
       [0.39407129, 0.24743099, 0.43091087, 0.23956816, 0.23797903]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


We can also use a threshold Brier Score, to score hits over a certain threshold.

In [18]:
threshold_brier_score = xs.threshold_brier_score(obs3, fct3, 0.7)
print(threshold_brier_score)

<xarray.DataArray (lat: 4, lon: 5)>
array([[0.11111111, 0.        , 0.        , 0.44444444, 0.11111111],
       [1.        , 1.        , 0.11111111, 0.11111111, 0.44444444],
       [0.        , 0.        , 0.11111111, 1.        , 0.11111111],
       [0.44444444, 0.11111111, 0.11111111, 1.        , 0.44444444]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


In [19]:
brier_score = xs.brier_score(obs3 > 0.5, (fct3 > 0.5).mean("member"))
print(brier_score)

<xarray.DataArray (lat: 4, lon: 5)>
array([[0.44444444, 0.11111111, 0.44444444, 0.        , 0.        ],
       [0.44444444, 0.11111111, 0.11111111, 0.44444444, 0.44444444],
       [0.        , 0.44444444, 1.        , 1.        , 0.44444444],
       [0.11111111, 0.11111111, 0.        , 1.        , 0.44444444]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


In [20]:
rank_histogram = xs.rank_histogram(obs3, fct3)
print(rank_histogram)

<xarray.DataArray 'histogram_rank' (rank: 4)>
array([8, 4, 5, 3])
Coordinates:
  * rank     (rank) float64 1.0 2.0 3.0 4.0


In [21]:
hist_event, hist_no_event = xs.discrimination(obs3 > 0.5, (fct3 > 0.5).mean("member"))
print(hist_event)

<xarray.DataArray (forecast_probability: 5)>
array([0.1, 0.3, 0. , 0.3, 0.3])
Coordinates:
  * forecast_probability  (forecast_probability) float64 0.1 0.3 0.5 0.7 0.9


In [22]:
print(hist_no_event)

<xarray.DataArray (forecast_probability: 5)>
array([0.1, 0.2, 0. , 0.5, 0.2])
Coordinates:
  * forecast_probability  (forecast_probability) float64 0.1 0.3 0.5 0.7 0.9


## Contingency-Based

To work with contingency-based scoring, first instantiate a `Contingency` object by passing in your observations, forecast, and observation/forecast bin edges. See https://www.cawcr.gov.au/projects/verification/#Contingency_table for more information.

In [23]:
dichotomous_category_edges = np.array([0, 0.5, 1]) # "dichotomous" mean two-category
dichotomous_contingency = xs.Contingency(obs, fct,
                                        dichotomous_category_edges,
                                        dichotomous_category_edges,
                                        dim=['lat','lon'])
dichotomous_contingency_table = dichotomous_contingency.table

In [24]:
print(dichotomous_contingency_table)

<xarray.DataArray 'histogram_observations_forecasts' (time: 3, observations_category: 2, forecasts_category: 2)>
array([[[7, 3],
        [4, 6]],

       [[2, 7],
        [2, 9]],

       [[4, 3],
        [8, 5]]])
Coordinates:
  * time                          (time) object 2000-01-01 00:00:00 ... 2000-...
    observations_category_bounds  (observations_category) <U10 '(0.0, 0.5]' '...
    forecasts_category_bounds     (forecasts_category) <U10 '(0.0, 0.5]' '(0....
  * observations_category         (observations_category) int64 1 2
  * forecasts_category            (forecasts_category) int64 1 2


Scores based on the constructed contingency table can be called via class methods. The available methods are:

* bias_score
* hit_rate
* false_alarm_ratio
* false_alarm_rate
* success_ratio
* threat_score
* equit_threat_score
* odds_ratio
* odds_ratio_skill_score
* accuracy
* heidke_score
* peirce_score
* gerrity_score

Below, we share a few examples of these in action.

In [25]:
print(dichotomous_contingency.bias_score())

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([0.9       , 1.45454545, 0.61538462])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


In [26]:
print(dichotomous_contingency.hit_rate())

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([0.6       , 0.81818182, 0.38461538])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


In [27]:
print(dichotomous_contingency.false_alarm_rate())

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([0.3       , 0.77777778, 0.42857143])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


In [28]:
print(dichotomous_contingency.odds_ratio_skill_score())

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([ 0.55555556,  0.125     , -0.09090909])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


Now we can leverage multi-category edges to make use of some scores.

In [29]:
multi_category_edges = np.array([0, 0.25, 0.75, 1])
multicategory_contingency = xs.Contingency(obs, fct,
                                           multi_category_edges,
                                           multi_category_edges,
                                           dim=['lat','lon'])

In [30]:
print(multicategory_contingency.accuracy())

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([0.35, 0.55, 0.25])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


In [31]:
print(multicategory_contingency.heidke_score())

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([-0.02362205,  0.24369748, -0.06382979])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


In [32]:
print(multicategory_contingency.peirce_score())

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([-0.02362205,  0.22834646, -0.06976744])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


In [33]:
print(multicategory_contingency.gerrity_score())

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([-0.06456044,  0.09340659, -0.07878788])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


## Accessors

You can also use `xskillscore` as a method of your `xarray` Dataset.

In [34]:
ds = xr.Dataset()
ds["obs_var"] = obs
ds["fct_var"] = fct

In the case that your Dataset contains both your observation and forecast variable, just pass them as strings into the function.

In [35]:
print(ds.xs.pearson_r("obs_var", "fct_var", "time"))

<xarray.DataArray (lat: 4, lon: 5)>
array([[-0.67103963, -0.7994918 , -0.4877047 ,  0.9009042 , -0.99576019],
       [ 0.01011846, -0.98098359,  0.99660269,  0.98889716,  0.16882124],
       [ 0.53017399, -0.27949033, -0.39054563, -0.62662994, -0.08972376],
       [-0.99835429,  0.44805023,  0.36821353,  0.99698985,  0.99991305]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


You can also pass in a separate Dataset that contains your observations or forecast variable.

In [36]:
ds = ds.drop_vars("fct_var")
print(ds.xs.pearson_r("obs_var", fct, "time"))

<xarray.DataArray (lat: 4, lon: 5)>
array([[-0.67103963, -0.7994918 , -0.4877047 ,  0.9009042 , -0.99576019],
       [ 0.01011846, -0.98098359,  0.99660269,  0.98889716,  0.16882124],
       [ 0.53017399, -0.27949033, -0.39054563, -0.62662994, -0.08972376],
       [-0.99835429,  0.44805023,  0.36821353,  0.99698985,  0.99991305]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4
